In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# stock_prices = pd.read_csv(os.path.join(dirname, "stock_prices.csv"))

train = pd.read_csv("train.csv")



In [3]:
train[["matched_size", "imbalance_size", "imbalance_buy_sell_flag"]]

train[train.matched_size == 0]

train["matched_ratio"] = train.imbalance_buy_sell_flag*train.imbalance_size/train.matched_size
# train["test_ratio"] = train.imbalance_buy_sell_flag*train.matched_size/train.imbalance_size

In [4]:
stock_id = train[train.matched_ratio.isna()].stock_id.unique()
train[train.stock_id.isin(stock_id)].seconds_in_bucket.unique()

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380,
       390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
       520, 530, 540])

here we can see that some stocks don't have an auction

In [5]:

def make_data(df, train=0, full_data=False):
	data = df.copy()
	# data = df.set_index(["stock_id", "date_id", "seconds_in_bucket"])

	data["imbalance_buy_sell"] = data.imbalance_size * data.imbalance_buy_sell_flag

	params = ["imbalance_buy_sell", "reference_price", "matched_size", "wap_logreturns", "matched_ratio"]

	# data["wap_logreturns"] = np.log(data.wap/data.wap.shift(1))

	data["prev_wap"] = data.groupby(["date_id", "stock_id"]).wap.shift(1)

	# data["prev_wap"] = data["prev_wap"].fillna(0)

	data["wap_logreturns"] = np.log(data.wap / data.prev_wap)

	data["matched_ratio"] = data.imbalance_buy_sell_flag*data.imbalance_size/data.matched_size

	# data["wap_man"] = data.eval("(bid_price*ask_size + ask_price*bid_size)/(bid_size + ask_size)")

	data["imb_s1"] = data.eval('(bid_size - ask_size)/(bid_size + ask_size)')
	data["imb_s2"] = data.eval('(imbalance_size - matched_size)/(matched_size + imbalance_size)')

	

	# data["matched_ratio"] = data["matched_ratio"].fillna(0)

	# data["wap_logreturns"] = data["wap_logreturns"].fillna(0)

	# data.replace([np.inf, -np.inf], np.nan, inplace=True)
	# data['ewa6'] = data.groupby(["stock_id","date_id"]).wap.transform(lambda x : x.ewm(span=6, adjust=False).mean()).values
	# data['ewa12'] = data.groupby(["stock_id","date_id"]).wap.transform(lambda x : x.ewm(span=12, adjust=False).mean()).values

	if full_data == True:
		return data

	data = data.fillna(0)
	features = ["wap_logreturns", "matched_ratio", "imb_s1", "imb_s2", "imbalance_buy_sell_flag"]
	if train == 0:
		X = data[features].to_numpy()
		y = data.target.to_numpy()
		# y = np.array(data.target).reshape(-1,1)
		return X[np.isfinite(X).all(1)], y[np.isfinite(X).all(1)]
		# return X, y

	X = data[features].copy()

	# X.wap_logreturns = X.wap_logreturns.fillna(0)
	# X.matched_ratio = X.matched_ratio.fillna(0)
	# X.fillna(0,inplace=True)



	return X.to_numpy()


In [6]:
df = make_data(train, full_data=True)

df[(df.matched_size - df.imbalance_size < 0)]

df.eval("(matched_size - imbalance_size)").abs()

0          10199673.95
1           1475610.34
2           1516488.16
3           6472063.35
4          17413064.99
              ...     
5237975    25839638.85
5237976     8838188.64
5237977    12725436.10
5237978    93772372.21
5237979    22189391.61
Length: 5237980, dtype: float64

In [7]:
X, y = make_data(train)

X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [8]:
model = LinearRegression()
model.fit(X_train,y_train)


LinearRegression()

In [9]:
model.coef_

predict_target = model.predict(X)

In [10]:
# train[train.seconds_in_bucket > 10].head()
# train.loc[train.stock_id==1,['target', 'time_id', 'stock_id', 'seconds_in_bucket']]
train[train.stock_id==1]['seconds_in_bucket'].unique()

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380,
       390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
       520, 530, 540])

In [11]:
pd.DataFrame(predict_target).describe()

,0
count,5.237980e+06
mean,-4.700479e-02
std,1.164468e+00
min,-2.444604e+01
25%,-9.335093e-01
50%,-4.088758e-02
75%,8.481626e-01
max,3.069427e+01


In [12]:

stats.describe(predict_target)



DescribeResult(nobs=5237980, minmax=(-24.446041803810186, 30.69426906777006), mean=-0.047004785990314775, variance=1.3559861573020044, skewness=-0.020295488676681576, kurtosis=-0.30614172276628526)

In [13]:
import optiver2023

env = optiver2023.make_env()
iter_test = env.iter_test()


In [14]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    X = make_data(test, train=1)
    sample_prediction['target'] = model.predict(X)
    # sample_prediction['target'] = 0.5
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [15]:
!tail submission.csv
# sub = pd.read_csv("./example_test_files/sample_submission.csv")
sub = pd.read_csv("submission.csv")

test = pd.read_csv("./example_test_files/test.csv")

revealed = pd.read_csv("./example_test_files/revealed_targets.csv")


# test.merge

480_540_190,1.0610191893325995
480_540_191,-0.3541315949536374
480_540_192,-0.6124456666407581
480_540_193,0.023973161389017027
480_540_194,0.9815181355444038
480_540_195,1.117856675182424
480_540_196,-0.9705855097864727
480_540_197,1.341672986093307
480_540_198,1.307507776236449
480_540_199,-0.15137476585857873


/tmp/ipykernel_888010/1503078690.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  sub = pd.read_csv("submission.csv")


In [16]:
sub = sub[sub.target != "target"]
sub.target = sub.target.astype(float)

In [17]:
ans = train[["row_id", "target"]].merge(sub, on="row_id", how="right")

In [18]:
mae = np.mean(np.abs(ans.target_x - ans.target_y))
mae

5.278030790748476

Original Naive Estimate: 5.27941

With bid-ask size imbalance: 5.279170945560908

As above but adding imbalance-matched difference ratio: 5.278630645573058